# カスタム訓練実装コード
 - mnist_cnn.py(kerasに付属していたサンプルコード)の改変
 - 独立版keras → tensorflowのkerasを利用するようにimport等変更
 - Sequential APIからSubclassing APIでモデル作成するように変更
 - 最後の方は理解に自信がない部分がある…

## メインにソースの参考としたページ
 - [Tensorflow2(Sequential API, Functional API, Subclassing API)とMNISTではじめる画像分類](https://qiita.com/hiro871_/items/8e8fd65c28d1e2a13fa9#-%E5%AD%A6%E7%BF%92%E6%96%B9%E6%B3%95%E3%83%86%E3%83%BC%E3%83%97%E3%81%AB%E3%82%88%E3%82%8B%E5%AD%A6%E7%BF%92)
 - [TensorFlow 2.X の使い方を VGG16／ResNet50 の実装と共に解説](https://qiita.com/anieca/items/9dfe3ef46e7b655bf3ee#%E3%82%AB%E3%82%B9%E3%82%BF%E3%83%A0%E8%A8%93%E7%B7%B4-%E5%AE%9F%E8%A3%85)

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [2]:
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D

class MnistCnnModel(keras.Model):
    def __init__(self):
        super(MnistCnnModel, self).__init__()
        self.conv1 = Conv2D(32, kernel_size=(3, 3), activation='relu')
        self.conv2 = Conv2D(64, (3, 3), activation='relu')
        self.pool1 = MaxPooling2D(pool_size=(2, 2))
        self.drop1 = Dropout(0.25)
        self.flat1 = Flatten()
        self.dens1 = Dense(128, activation='relu')
        self.drop2 = Dropout(0.5)
        self.dens2 = Dense(num_classes, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.pool1(x)
        x = self.drop1(x)
        x = self.flat1(x)
        x = self.dens1(x)
        x = self.drop2(x)
        return self.dens2(x)

model = MnistCnnModel()
# model.summary()

In [3]:
# model.compile(loss=tf.keras.losses.categorical_crossentropy,
#               optimizer=tf.keras.optimizers.Adadelta(),
#               metrics=['accuracy'])

loss_obj = tf.keras.losses.CategoricalCrossentropy(reduction='none')
optimizer_obj = tf.keras.optimizers.Adadelta()

metrics_train = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')
train_loss = tf.keras.metrics.Mean(name='train_loss')
metrics_test = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')

### model.compileを自力ソースに変更する場合の理解

#### lossについて
 - kerasでは、lossにはfunctionとclassの呼び方2つがある
   - function → snake (e.g. `tf.keras.losses.categorical_crossentropy`)
   - class → upper camel (e.g. `tf.keras.losses.CategoricalCrossentropy()`)
 - classだとコンストラクタでreductionが指定できる
   - [keras公式ドキュメント](https://keras.io/api/losses/#Standalone-usage-of-losses)の例を見た感じ、<br/>
   `reduction='none'`の場合、functionの呼び方でlossを指定した場合と同じ結果を返すと思われる

#### metricsについて
 - compile関数で`metrics=['accuracy']`とした場合、自動で評価関数を何とすべきか判断してくれていた
   - `metrics=[tf.keras.metrics.CategoricalAccuracy()]`のような書き方もできる
 - カスタム訓練の場合で、カスタムじゃない場合と同じ用に値を記録するには<br/>
 lossとaccuracyで2個、さらにtrain用とtest用で2個で4つオブジェクトが必要っぽい


In [4]:
# model.fit(x_train, y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           verbose=1,
#           validation_data=(x_test, y_test))

# 訓練用関数の定義　@tf.functionは処理を早くするために書くやつ
@tf.function
def train_step(x, t):
    with tf.GradientTape() as tape:
        predictions = model(x, training=True)
        loss = loss_obj(t, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer_obj.apply_gradients(zip(gradients, model.trainable_variables))

    metrics_train(t, predictions)
    train_loss(loss)

# テスト用関数の定義
@tf.function
def test_step(x, t):
    predictions = model(x)
    loss = loss_obj(t, predictions)

    metrics_test(t, predictions)
    test_loss(loss)

# 学習データ準備のコード
buffer_size = len(x_train)
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(buffer_size).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

# 実際の訓練やテストを実行する(指定したepoch分))
for epoch in range(epochs):

    # 訓練してテストする
    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    # ログ出す
    txt = 'Epoch {}/{} loss: {} - accuracy: {} val_loss: {} - val_accuracy: {}'
    print(txt.format(
        "{:2d}".format(epoch + 1),
        epochs,
        "{:1.4f}".format(train_loss.result()),
        "{:1.4f}".format(metrics_train.result()),
        "{:1.4f}".format(test_loss.result()),
        "{:1.4f}".format(metrics_test.result())
    ))

    # 評価をリセットする リセットしないとすべてのepochでの評価になる？
    metrics_train.reset_states()
    train_loss.reset_states()
    metrics_test.reset_states()
    test_loss.reset_states()

# score = model.evaluate(x_test, y_test, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])
# model.summary()


Epoch  1/12 loss: 2.2635 - accuracy: 0.1934 val_loss: 2.2016 - val_accuracy: 0.3889
Epoch  2/12 loss: 2.1545 - accuracy: 0.3414 val_loss: 2.0606 - val_accuracy: 0.5050
Epoch  3/12 loss: 2.0018 - accuracy: 0.4498 val_loss: 1.8638 - val_accuracy: 0.6116
Epoch  4/12 loss: 1.7972 - accuracy: 0.5382 val_loss: 1.6099 - val_accuracy: 0.6857
Epoch  5/12 loss: 1.5645 - accuracy: 0.5977 val_loss: 1.3408 - val_accuracy: 0.7392
Epoch  6/12 loss: 1.3458 - accuracy: 0.6403 val_loss: 1.1053 - val_accuracy: 0.7770
Epoch  7/12 loss: 1.1705 - accuracy: 0.6745 val_loss: 0.9278 - val_accuracy: 0.8034
Epoch  8/12 loss: 1.0440 - accuracy: 0.6985 val_loss: 0.8029 - val_accuracy: 0.8221
Epoch  9/12 loss: 0.9520 - accuracy: 0.7214 val_loss: 0.7139 - val_accuracy: 0.8348
Epoch 10/12 loss: 0.8783 - accuracy: 0.7370 val_loss: 0.6478 - val_accuracy: 0.8469
Epoch 11/12 loss: 0.8269 - accuracy: 0.7500 val_loss: 0.5980 - val_accuracy: 0.8550
Epoch 12/12 loss: 0.7824 - accuracy: 0.7620 val_loss: 0.5585 - val_accuracy:

### model.fitを自力ソースに変更する場合の理解

#### 訓練の流れ　たぶんこんな感じ
 - `training=True`にして訓練データをモデルにぶっこむ
 - 損失を計算する
 - **勾配を計算する**
 - **勾配に従って値を更新する**
 - 評価する（accuracy、loss）
#### テストの流れ　たぶんこんな感じ
 - `training=False`(指定しなければデフォルトの値)にして訓練データをモデルにぶっこむ
 - 損失を計算する
 - 評価する(accuracy、loss）

